In [ ]:
from nerfstudio.data.datamanagers.base_datamanager import VanillaDataManagerConfig, VanillaDataManager



In [2]:
from nerfstudio.engine.trainer import TrainerConfig
from nerfstudio.engine.optimizers import AdamOptimizerConfig
from nerfstudio.engine.schedulers import ExponentialDecaySchedulerConfig
from nerfstudio.data.dataparsers.nerfstudio_dataparser import NerfstudioDataParserConfig
from nerfstudio.data.dataparsers.blender_dataparser import BlenderDataParserConfig
from nerfstudio.pipelines.dynamic_batch import DynamicBatchPipelineConfig
from nerfstudio.models.instant_ngp import InstantNGPModelConfig
from nerfstudio.configs.base_config import ViewerConfig
from pathlib import Path

trainer_config = TrainerConfig(
    method_name="instant-ngp",
    steps_per_eval_batch=500,
    steps_per_save=2000,
    max_num_iterations=30000,
    mixed_precision=True,
    pipeline=DynamicBatchPipelineConfig(
        datamanager=VanillaDataManagerConfig(
            dataparser=BlenderDataParserConfig(),
        ),
        model=InstantNGPModelConfig(eval_num_rays_per_chunk=8192),
    ),
    optimizers={
        "fields": {
            "optimizer": AdamOptimizerConfig(lr=1e-2, eps=1e-15),
            "scheduler": ExponentialDecaySchedulerConfig(lr_final=0.0001, max_steps=200000),
        }
    },
    viewer=ViewerConfig(num_rays_per_chunk=1 << 12),
    vis="viewer",
)

trainer_config.vis = "viewer"
trainer_config.pipeline.datamanager.data = Path("../data/nerf_synthetic/lego")
trainer_config.set_timestamp()
trainer_config.save_config()

trainer = trainer_config.setup(local_rank=0, world_size=1)
trainer.setup()
trainer.train()

(viser) Connection opened (4, 1 total), 619 persistent messages

(viser) Connection closed (4, 0 total)

KeyboardInterrupt: 